In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

import cv2
import numpy as np

In [3]:
ref_images = !ls 210226_Bladder_TMA1_reg36/1_shading_correction/*.tif
fixed_images = !ls 210226_Bladder_TMA1_reg36/4_stitching/*.tif

ref_file = ref_images[3]
fixed_file = fixed_images[3]
print(ref_file)
print(fixed_file)

210226_Bladder_TMA1_reg36/1_shading_correction/210226_Bladder_TMA1_reg36_cycle01_channel00_DAPI_4.tif
210226_Bladder_TMA1_reg36/4_stitching/210226_Bladder_TMA1_reg36_cycle01_channel00_DAPI_4.tif


In [6]:
ref_image = cv2.imread(ref_file, -1)
print(ref_image.shape)
q = np.quantile(ref_image, 0.9999)
ref_image[ref_image>q] = q
ref_image = (255*(ref_image/q)).astype(np.uint8)

(4096, 4096)


In [7]:
fixed_image = cv2.imread(fixed_file, -1)
print(fixed_image.shape)
q = np.quantile(fixed_image, 0.9999)
fixed_image[fixed_image>q] = q
fixed_image = (255*(fixed_image/q)).astype(np.uint8)

(3891, 3891)


In [90]:
cv2.imwrite('z.png', fixed_image)

True

In [54]:
width = 2048
overlap_pct = 0.1
overhang = 100
overlap_size = int(np.ceil(width * overlap_pct))
print(overlap_size)

205


In [51]:
# x1 = ref_image[:, 2480-overlap_size*2-overhang:2048+overhang]
# x1 = x1[512:1536,:]
marked = ref_image.copy()
marked[:, 2046:2050] = 255
marked[2046:2050, :] = 255
z = 2046-overlap_size
marked[z:z+2, :] = 255
marked[:, z:z+2] = 255
z = 2046+overlap_size
marked[z:z+2, :] = 255
marked[:, z:z+2] = 255
cv2.imwrite('figures/stitching/x.png', marked)

True

In [105]:
p1 = 2400
p2 = 3600
b1 = ref_image[2048-overlap_size-overhang:2048, p1:p2]
b2 = ref_image[2048:2048+overlap_size+overhang, p1:p2]
b3 = fixed_image[2048-overlap_size+8:2048-overlap_size+8+overlap_size+overhang, p1-overlap_size+15:p2-overlap_size+15]

# print(border_1.shape, border_2.shape, border_3.shape)
# b1 = border_1[512:1536,:]
# b2 = border_2[512:1536,:]
# b3 = border_3[512:1536,:]
b1 = np.concatenate([b1, np.zeros((overhang, b1.shape[1]), dtype=b1.dtype)], axis=0)
b2 = np.concatenate([np.zeros((overhang, b2.shape[1]), dtype=b2.dtype), b2], axis=0)
b3 = np.concatenate([np.zeros((overhang, b3.shape[1]), dtype=b3.dtype), b3], axis=0)
print(b1.shape, b2.shape, b3.shape)

cv2.imwrite('figures/stitching/b1.png', b1)
cv2.imwrite('figures/stitching/b2.png', b2)
cv2.imwrite('figures/stitching/b3.png', b3)

rgb_before = np.dstack([b1, b2, np.zeros_like(b1)])
rgb_after = np.dstack([b1, b3, np.zeros_like(b1)])

cv2.imwrite('figures/stitching/overlap_before.png', rgb_before[:,:,::-1])
cv2.imwrite('figures/stitching/overlap_fixed.png', rgb_after[:,:,::-1])

(405, 1200) (405, 1200) (405, 1200)


True